# Tutorial: spectral_utils.py

This notebook demonstrates how to use the methods in the spectral_utils.py utility. There are four classes (SHT, Fourier, Legendre, and Chebyshev) that are used to perform their respective transformations between physical and spectral space, and to perform angular and radial derivatives.

# Contents
1.  [Generate sample data](#sample)
2.  [Spherical harmonic transforms](#SHT)
    1. [Convert Shell Spectra to Shell Slices](#case-1)
    2. [Convert Shell Slices to Shell Spectra](#case-2)
    3. [Legendre transform on Shell Spectra](#case-3)
    4. [Fourier transform on Shell Slices](#case-5)
    5. [Fourier transform on Shell Spectra](#case-7)
3.  [Angular derivatives](#derivs)
    1. [$\sin \theta \frac{\partial}{\partial\theta}$ in spectral space](#case-9)
    2. [$\frac{\partial}{\partial \phi}$ in spectral space using the SHT class](#case-10)
    3. [$\frac{\partial}{\partial \phi}$ in spectral space using the Fourier class](#case-11)
    4. [$\frac{\partial}{\partial \theta}$ in physical space](#case-12)
    5. [$\frac{\partial}{\partial \phi}$ in physical space](#case-13)
4.  [Radial transforms and derivatives](#radial)
    1. [Chebyshev transform](#cheby)
    2. [Radial derivatives](#ddr)



In [ ]:
# import the relevant utility
import spectral_utils

# import Shell Spectra/Slices classes from Rayleigh
from rayleigh_diagnostics import Shell_Spectra, Shell_Slices

# import other helpful things
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import sph_harm
from scipy.special import chebyt

# plotting nonsense
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm,colors
plt.rcParams['image.cmap'] = 'seismic'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.interpolation'] = 'none'

<a id="sample"></a>
# 1. Generate Sample Shell Slice and Sample Shell Spectrum

If fakedata=True, a sample shell slice and shell spectrum are generated using a combination of spherical harmonics. If you would prefer to use your own Rayleigh outputs, set fakedata=False and set the "sample_slice_name", "sample_slice_path, "sample_spectrum_name", and "sample_spectrum_path" to the relevant values.

In [ ]:
fakedata = True
sample_slice_name = 'sample_slice'
sample_spectrum_name = 'sample_spectrum'
sample_slice_path = ''
sample_spectrum_path = ''

# generate fake spherical harmonic data 
if fakedata:
    
    #parameters
    ntheta = 384
    dealias = 1.5
    lmax = int(ntheta/dealias - 1)
    nphi = 768
    
    # Create 2D grid of angular variables
    theta = np.linspace(0, np.pi, ntheta)
    phi = np.linspace(0, 2*np.pi, nphi)
    phi, theta = np.meshgrid(theta, phi)

    # generate spherical harmonics with l,m = (2,2), (5, 3), and (8, 5) 
    Ylm_1  = sph_harm(2, 2, theta, phi)
    Ylm_2 = sph_harm(3, 5, theta, phi)
    Ylm_3 = sph_harm(5, 8, theta, phi)
    sample_slice = Ylm_1 + Ylm_2 + Ylm_3
    sample_slice = np.real(np.transpose(sample_slice))
    
    # take the spherical harmonic transform for future examples
    transform_SHT = spectral_utils.SHT(ntheta, spectral=False)
    sample_spectrum = np.zeros((lmax+1, lmax+1), dtype=complex)
    sample_spectrum = transform_SHT.to_spectral(sample_slice, th_l_axis=0, phi_m_axis=1)
    xlim=10
    norm=None
    

else:
    # read in sample shell slice 
    sslice = Shell_Slices(sample_slice_name, path=sample_slice_path)
    sample_slice = sslice.vals[:,:,-1,sslice.lut[301],0] # radial vorticity spectrum (301) at the bottom of the radiative interior
    sample_slice = np.transpose(sample_slice)
    
    # read in sample shell spectrum (corresponding to above shell slice)
    ss = Shell_Spectra(sample_spectrum_name, path=sample_spectrum_path)
    lmax = ss.lmax
    sample_spectrum = np.zeros((lmax+1, lmax+1), dtype=complex)
    sample_spectrum = ss.vals[:,:,-1,ss.lut[301],0]    
    
    # parameters
    ntheta = sslice.ntheta
    dealias = ntheta/(lmax + 1)
    nphi = sslice.nphi
    
    xlim=50
    norm=colors.LogNorm(vmin=1e-30, vmax=1e-15)

#    
# plot the sample slice and sample spectrum
fig = plt.figure(figsize=(14, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1]) 

ax0 = plt.subplot(gs[0])
ax0.set_title('Sample Shell Slice')
pos0=ax0.imshow(sample_slice, extent=[0, 360, -90, 90])
ax0.set_xlabel('Longitude')
ax0.set_ylabel('Latitude')
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos0, cax=cax)

ax1 = plt.subplot(gs[1])
ax1.set_title('Sample Shell Spectra')
power = sample_spectrum.real**2 + sample_spectrum.imag**2
pos1=ax1.imshow(np.transpose(power), norm=norm)
ax1.set_xlabel('l')
ax1.set_ylabel('m')
ax1.set_xlim(0, xlim)
ax1.set_ylim(0, xlim)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos1, cax=cax)

<a id="SHT"></a>
# 2. Spherical harmonic transformations

The SHT class performs full spherical harmonic transformations with both the Legendre and Fourier transforms, while the Fourier and class performs only the Fourier transform in $m$. Each class features to_physical and to_spectral functions. Below is a table denoting the currently supported transformations and the best class and function to perform each:

| link | input | output | transformations | class | function | use case
| --- | --- | --- | --- | --- | --- | ---
| [Case 1](#case-1)| $l$, $m$         | $\theta$, $\phi$ | iLT + iFFT      | SHT | to_physical(data, th_l_axis=0, phi_m_axis=1) | convert Shell Spectra to Shell Slices
| [Case 2](#case-2) | $\theta$, $\phi$ | $l$, $m$         | FFT + LT        | SHT | to_spectral(data, th_l_axis=0, phi_m_axis=1) | convert Shell Slices to Shell Spectra
| [Case 3](#case-3) | $l$, $m$         | $\theta$, $m$    | iLT             | SHT | _LT_to_physical(data, axis=0, m_axis=1) | perform inverse Legendre transform of Shell Spectra 
| [Case 4](#case-3) | $\theta$,    $m$ | $l$, $m$         | LT              | SHT | _LT_to_spectral(data, axis=0, m_axis=1) | reverse of Case 3
| [Case 5](#case-5) | $\theta$, $\phi$ | $\theta$, $m$    | FFT             | Fourier | to_spectral(data, axis=0, window=None) | perform Fourier transform of Shell Slice
| [Case 6](#case-5) | $\theta$,    $m$ | $\theta$, $\phi$ | iFFT            | Fourier | to_physical(data, axis=0) | reverse of Case 5
| [Case 7](#case-7) | $l$, $m$         | $l$, $\phi$      | iFFT            | Fourier | to_physical(data, axis=0) | perform inverse Fourier transform of Shell Spectra
| [Case 8](#case-7) | $l$, $\phi$      | $l$, $m$         | FFT             | Fourier | to_spectral(data, axis=0, window=None) | reverse of Case 7


<a id="case-1"></a>
## A. Case 1: Convert Shell Spectra to Shell Slices,  $(l, m) \rightarrow (\theta, \phi)$
### Instantiate class

In [ ]:
# instantiate SHT class
SHT = spectral_utils.SHT(ntheta, spectral=False, dealias=dealias)

# alternate instantiation
# SHT = spectral_utils.SHT(lmax, spectral=True, dealias=dealias)

### Perform transformation

In [ ]:
# convert from spectral space to physical space
sample_spectra_to_slice = SHT.to_physical(sample_spectrum, th_l_axis=0, phi_m_axis=1)

### Plot results

In [ ]:
# plot and compare the transformed spectra to the matching slice
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 8))

pos0 = ax0.imshow(sample_spectra_to_slice, extent=[0, 360, -90, 90])
ax0.set_title('Converted Data')
ax0.set_xlabel('Longitude')
ax0.set_ylabel('Latitude')

divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos0, cax=cax)


ax1.set_title('Original Slice')
pos1 = ax1.imshow(sample_slice.real, extent=[0, 360, -90, 90])
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos1, cax=cax)

<a id="case-2"></a>
## B. Case 2: Convert Shell Slices to Shell Spectra, $(\theta, \phi) \rightarrow (l, m)$
### Instantiate class

In [ ]:
# instantiate SHT class
SHT = spectral_utils.SHT(ntheta, spectral=False, dealias=dealias)

# alternate instantiation
# SHT = spectral_utils.SHT(lmax, spectral=True, dealias=dealias)

### Perform transformation

In [ ]:
# this transformation will produce a complex output, so initialize a complex array
sample_slice_to_spectra = np.zeros((lmax+1, lmax+1), dtype=complex)

# transform from physical space to spectral space
sample_slice_to_spectra = SHT.to_spectral(sample_slice, th_l_axis=0, phi_m_axis=1)

### Plot results

In [ ]:
# calculate power
power = sample_slice_to_spectra.real**2 + sample_slice_to_spectra.imag**2

# plot the spectra to see how they look
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(13, 5))

ax0.set_title('Converted Data')
pos0=ax0.imshow(np.transpose(power), norm=norm)
ax0.set_xlabel('l')
ax0.set_ylabel('m')
ax0.set_xlim(0, xlim)
ax0.set_ylim(0, xlim)
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos0, cax=cax)

power = sample_spectrum.real**2 + sample_spectrum.imag**2

ax1.set_title('Original Spectrum')
pos1=ax1.imshow(np.transpose(power), norm=norm)
ax1.set_xlabel('l')
ax1.set_ylabel('m')
ax1.set_xlim(0, xlim)
ax1.set_ylim(0, xlim)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos1, cax=cax)

<a id="case-3"></a>
## C. Cases 3 and 4: Perform Legendre transform on Shell Spectra $(l, m) \leftrightarrow (\theta, m)$
### Instantiate class

In [ ]:
# instantiate SHT class
SHT = spectral_utils.SHT(ntheta, spectral=False, dealias=dealias)

# alternate initialization
# SHT = spectral_utils.SHT(lmax, spectral=True, dealias=dealias)

### Perform inverse Legendre transformation $(l, m) \rightarrow (\theta, m)$

In [ ]:
sample_spectra_iLT = np.zeros((ntheta, lmax+1), dtype=complex)
sample_spectra_iLT = SHT._LT_to_physical(sample_spectrum, axis=0, m_axis=1)

### Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

pos = ax.imshow(sample_spectra_iLT.real,extent=[-0.5, np.shape(sample_spectra_iLT)[1] - 0.5, -90, 90], aspect='auto',
#               vmin=-0.5e-9, vmax=0.5e-9
               )
ax.set_xlim(-0.5, 10)
ax.set_xlabel('m')
ax.set_ylabel('Latitude')
fig.colorbar(pos, ax=ax)

### Perform Legendre transformation $(\theta, m) \rightarrow (l, m)$

In [ ]:
sample_spectra_LT = SHT._LT_to_spectral(sample_spectra_iLT*nphi, axis=0, m_axis=1)

### Plot results

In [ ]:
power = sample_spectra_LT.real**2 + sample_spectra_LT.imag**2

fig, ax = plt.subplots(figsize=(8, 5))

pos = ax.imshow(np.transpose(power), norm=norm)
ax.set_xlim(0, xlim)
ax.set_ylim(0, xlim)
ax.set_xlabel('l')
ax.set_ylabel('m')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

<a id="case-5"></a>
## D. Cases 5 and 6: Perform Fourier transform on Shell Slice $(\theta, \phi) \leftrightarrow (\theta, m)$
### Instantiate class

In [ ]:
fourier = spectral_utils.Fourier(nphi)

### Perform Fourier transformation $(\theta, \phi) \rightarrow (\theta, m)$

In [ ]:
sample_slice_FT = fourier.to_spectral(sample_slice, axis=1)

### Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

pos = ax.imshow(sample_slice_FT.real, aspect='auto', extent=[-0.5, np.shape(sample_slice_FT)[0]-0.5, -90, 90],
#                vmin=-1e-9, vmax=1e-9
         )
ax.set_xlim(-0.5, 10)
ax.set_xlabel('m')
ax.set_ylabel('Latitude')
fig.colorbar(pos, ax=ax)

### Perform inverse Fourier transformation $(\theta, m) \rightarrow (\theta, \phi)$

In [ ]:
sample_slice_iFFT = fourier.to_physical(sample_slice_FT, axis=1)

### Plot results

In [ ]:
# plot the power to see if it makes sense
fig, ax = plt.subplots(figsize=(8, 5))
pos = ax.imshow(sample_slice_iFFT.real, extent=[0, 360, -90, 90])
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

<a id="case-7"></a>
## E. Cases 7 and 8: Perform Fourier transform on Shell Spectra $(l, m) \leftrightarrow (l, \phi)$
### Instantiate class
Because we are starting from fully transformed spherical harmonic spectra, the number of angular values is set by lmax*2 + 1, rather than nphi.

In [ ]:
fourier = spectral_utils.Fourier(lmax*2 + 1)

### Perform inverse Fourier transformation $(l, m) \rightarrow (l, \phi)$

In [ ]:
sample_spectra_iFFT = fourier.to_physical(sample_spectrum, axis=1)

### Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

pos = ax.imshow(sample_spectra_iFFT, aspect='auto',extent=[-0.5, 360, -0.5, lmax-0.5],
#               vmin=-1e-9, vmax=1e-9
               )
ax.set_ylim(0.5, 15)
ax.set_xlabel('Longitude')
ax.set_ylabel('l')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

### Perform Fourier transformation $(l, \phi) \rightarrow (l, m)$

In [ ]:
# reinstantiate class with the correct number of grid points
fourier = spectral_utils.Fourier(lmax*2)
sample_spectra_FT = fourier.to_spectral(sample_spectra_iFFT, axis=1)

### Plot results

In [ ]:
power = sample_spectra_FT.real**2 + sample_spectra_FT.imag**2

fig, ax = plt.subplots(figsize=(8, 5))

pos = ax.imshow(np.transpose(power), norm=norm)
ax.set_xlim(0, xlim)
ax.set_ylim(0, xlim)
ax.set_xlabel('l')
ax.set_ylabel('m')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

<a id="derivs"></a>
# 3. Angular derivatives

The SHT, Fourier, and Legendre classes can perform angular derivatives in both spectral and physical space. A summary of the available functions is provided in the table below:

| derivative | domain | class | function
| --- | --- | --- | ---
|[$\sin \theta \frac{\partial}{\partial \theta}$](#case-9) | spectral | SHT      | sin_d_dtheta(data, l_axis=0, m_axis=1)
|[$\frac{\partial}{\partial \phi}$](#case-10)             | spectral | SHT      | d_dphi(data, m_axis=0)
|[$\frac{\partial}{\partial \phi}$](#case-11)             | spectral | Fourier  | d_dphi(data, axis=0, physical=False)
|[$\frac{\partial}{\partial \theta}$](#case-12)           | physical | Legendre | d_dtheta(data, axis=0, physical=True)
|[$\frac{\partial}{\partial \phi}$](#case-13)             | physical | Fourier  | d_dphi(data, axis=0, physical=True)

<a id="case-9"></a>
## A. $\sin \theta \frac{\partial}{\partial\theta}$ in spectral space
### Instantiate class

In [ ]:
SHT = spectral_utils.SHT(ntheta, spectral=False, dealias=dealias)

### Take derivative

In [ ]:
l_derivative = SHT.sin_d_dtheta(sample_spectrum, l_axis=0, m_axis=1)

### Plot results

In [ ]:
power = l_derivative.real**2 + l_derivative.imag**2

fig, ax = plt.subplots(figsize=(8, 5))

pos=ax.imshow(np.transpose(power), norm=norm)
ax.set_xlim(0, xlim)
ax.set_ylim(0, xlim)
ax.set_xlabel('l')
ax.set_ylabel('m')
fig.colorbar(pos, ax=ax)

<a id="case-10"></a>
## B. $\frac{\partial}{\partial \phi}$ in spectral space using the SHT class

In [ ]:
m_derivative = SHT.d_dphi(sample_spectrum, m_axis=1)

In [ ]:
power = m_derivative.real**2 + m_derivative.imag**2

fig, ax = plt.subplots(figsize=(8, 5))

pos=ax.imshow(np.transpose(power), norm=norm)
ax.set_xlim(0, xlim)
ax.set_ylim(0, xlim)
ax.set_xlabel('l')
ax.set_ylabel('m')
fig.colorbar(pos, ax=ax)

<a id="case-11"></a>
## C. $\frac{\partial}{\partial \phi}$ in spectral space using the Fourier class
### Instantiate class

In [ ]:
Fourier = spectral_utils.Fourier(lmax*2 + 1)

### Take derivative

In [ ]:
m_derivative = Fourier.d_dphi(sample_spectrum, axis=1, physical=False)

### Plot results

In [ ]:
power = m_derivative.real**2 + m_derivative.imag**2

fig, ax = plt.subplots(figsize=(8, 5))

pos=ax.imshow(np.transpose(power), norm=norm)
ax.set_xlim(0, xlim)
ax.set_ylim(0, xlim)
ax.set_xlabel('l')
ax.set_ylabel('m')
fig.colorbar(pos, ax=ax)

<a id="case-12"></a>
## D. $\frac{\partial}{\partial \theta}$ in physical space
### Instantiate class

In [ ]:
Legendre = spectral_utils.Legendre(ntheta, spectral=False, dealias=dealias)

# alternate instantiation
# Legendre_class = spectral_utils.Legendre(lmax, spectral=True, dealias=dealias)

### Take derivative

In [ ]:
th_derivative = Legendre.d_dtheta(sample_slice, axis=0, physical=True)

### Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

pos=ax.imshow(th_derivative, extent=[0, 360, -90, 90])
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

<a id="case-13"></a>
## E. $\frac{\partial}{\partial \phi}$ in physical space
### Instantiate class

In [ ]:
Fourier = spectral_utils.Fourier(nphi)

### Take derivative

In [ ]:
ph_derivative = Fourier.d_dphi(sample_slice, axis=1, physical=True)

### Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

pos=ax.imshow(ph_derivative, extent=[0, 360, -90, 90])
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='2%', pad=0.05)
fig.colorbar(pos, cax=cax)

<a id="radial"></a>
# 4. Radial transformations and derivatives
The Chebyshev class provides methods to transform between physical and spectral space and perform radial derivatives in physical and spectral space. This example generates sample Chebyshev polynomials for simplicity, but this class can be used on any data output on the full radial grid.

### Instantiate class
This class currently supports three different types of grids: a) single Chebyshev domain, b) uniform set of N Chebyshev domains, and c) N Chebyshev domains with different resolutions. This example uses a single Chebyshev domain with $n_r = 128$. Instantiation examples of other grid types are listed in the comment below. 

In [ ]:
# set rmin and rmax for our fake data
rmin=5e10
rmax=6.83177e10

# 
cheb = spectral_utils.Chebyshev(128, rmin=rmin, rmax=rmax)

# Single Chebyshev domain with 72 grid points using shell depth & aspect ratio:
# >>> cheb = Chebyshev(72, aspect_ratio=0.2, shell_depth=2)
# >>> cheb.rmin, cheb.rmax
# (0.5, 2.5)

# Same grid as before, but specifying the minimum/maximum radius:
# >>> cheb = Chebyshev(72, rmin=0.5, rmax=2.5)
# >>> cheb.rmin, cheb.rmax
# (0.5, 2.5)

# Same grid as before, but specifying the boundaries:
# >>> cheb = Chebyshev(72, boundaries=(0.5, 2.5))
# >>> cheb.rmin, cheb.rmax
# (0.5, 2.5)
# >>> cheb.nr_domains
# [72]

# Three Chebyshev domains, each with 24 grid points, 72 grid points total:
# >>> cheb = Chebyshev(24, n_uniform_domains=3, aspect_ratio=0.2, shell_depth=2)
# >>> cheb.boundaries
# [2.5, 1.83333, 1.16666, 0.5]
# >>> cheb.nr_domains
# [24, 24, 24]

# Three Chebyshev domains, nonuniform resolutions, 72 grid points total:
# >>> cheb = Chebyshev([16,36,20], boundaries=[0.5,1.0,2.0,2.4])
# >>> cheb.boundaries
# [2.5, 2.0, 1.0, 0.5]
# >>> cheb.nr_domains
# [20, 36, 16]

### Generate sample data

In [ ]:
# generate radial grid
radius = cheb.radius

# generate colocation points in the [-1, 1] range
x = cheb.x

# generate data from a handful of Chebyshev polynomials
rdata = chebyt(2)(x) - 4*chebyt(4)(x) + 3*chebyt(10)(x)

plt.plot(radius, rdata)
plt.xlabel('Radius')
plt.title('Sample radial data')

<a id="cheby"></a>
## A. Chebyshev transform
### Transform from physical to spectral space

In [ ]:
to_spectral = cheb.to_spectral(rdata, axis=0)

### Plot results

In [ ]:
# plot the decomposition
reconstructed = np.zeros(np.shape(to_spectral))
reconstructed = to_spectral[0]*chebyt(0)(x)/2    # the coefficient for n=0 is 1/N, as opposed to 2/N

for i in range(1, len(to_spectral)):
    reconstructed += to_spectral[i]*chebyt(i)(x)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.8))

ax1.set_title('Spectral coefficients')
ax1.plot(to_spectral)
ax1.set_xlim(-0.5, 15)
ax1.set_xlabel('Order')
ax1.set_ylabel('Coefficient')

ax2.set_title('Comparison')
ax2.plot(radius, rdata, label='Original')
ax2.scatter(radius, reconstructed, label='Reconstructed', color='orange')
ax2.legend()

### Transform from spectral to physical space

In [ ]:
to_physical = cheb.to_physical(to_spectral, axis=0)

In [ ]:
plt.plot(radius, rdata, label='original')
plt.scatter(radius, to_physical, label='transformed', color='orange')
plt.legend()

<a id="ddr"></a>
## B. Radial derivatives
### $\frac{\partial}{\partial r}$ in physical space

In [ ]:
ddr_physical = cheb.d_dr(rdata, physical=True)

In [ ]:
plt.plot(radius, ddr_physical, label='Chebyshev d_dr')
plt.scatter(radius, np.gradient(rdata[:,0], radius), label='numpy gradient', color='orange')
plt.legend()

### $\frac{\partial}{\partial r}$ in spectral space

In [ ]:
ddr_spectral = cheb.d_dr(to_spectral, physical=False)

In [ ]:
plt.plot(ddr_spectral)
plt.xlim(0, 15)
plt.xlabel('Order')
plt.ylabel('Coefficient')

In [ ]:
# transform back and plot against the physical derivative to compare
ddr_spectral_to_physical = cheb.to_physical(ddr_spectral, axis=0)

plt.plot(radius, ddr_physical, label='physical space')
plt.scatter(radius, ddr_spectral_to_physical, color='orange', label='spectral space')
plt.legend()